In [2]:
from transformers import AutoTokenizer,AutoModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import os
import json
import re
import torch
from decimal import Decimal
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#cuda_DEVICE = ['cuda:3','cuda:4']

def load_model(which_model):
    if which_model == 'model1': # 原始模型
        model_path = '/HL_user01/llm_models/chatglm3-6b'
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        model = AutoModel.from_pretrained(model_path, trust_remote_code=True,device='cuda:1')
    elif which_model == 'model_wjc': # 全科室模型
        model_path = '/HL_user01/trained_models/0229_ck36000_sft_stage4_lora_03-27-09-27-27_export_model'
        # model_path = '/data/wangjiacheng/瑞金/1228_测试/export_models/chuyuanxiaojie_1201'
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        model = AutoModel.from_pretrained(model_path,torch_dtype=torch.float16, trust_remote_code=True,device='cuda:1')
    elif which_model == 'model_ht': # 华佗
        model_path = '/HL_user01/medical_llm/HuatuoGPT2-7b'
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        model = AutoModel.from_pretrained(model_path,torch_dtype=torch.float16, trust_remote_code=True)
        
    model = model.eval()
    return model,tokenizer

#model_path = '/HL_user01/llm_models/chatglm3-6b'
#model_path = '/HL_user01/lyh/trained_model/all_cyxjs_chatglm3_lora_lyh627/checkpoint-2000'
# tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)
# model = AutoModel.from_pretrained(model_path,trust_remote_code=True).cuda()
# model = model.eval()

which_model='model_wjc'
model,tokenizer=load_model(which_model)



/HL_user01/conda/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.03s/it]


In [78]:
import re
jiancha_source='''
##日常病程记录
时间:2019-03-19
文本:患者今日超声引导下行右侧腋窝及锁骨上淋巴结细针穿刺术。 影像描述:A:于右侧腋窝可见几个低回声，之一大小约 24.7×9.3mm。淋巴结间未见相互融合，包膜光整，与周围软组织分界清，外形呈椭圆形，淋巴结淋巴门偏心移位，淋巴结内部呈低回声(与毗邻肌肉相比较)，分布较均，未见明显钙化强回声，未见明显液化无回声。CFI可显示淋巴结淋巴门血管，较丰富。 B:于右侧锁骨上可见几个低回声，之一大小约7.2×5.0 mm。淋巴结间未见相互融合，包膜光整，与周围软组织分界清，外形趋圆，淋巴结淋巴门不明显，淋巴结内部呈低回声(与毗邻肌肉相比较)，分布较均，未见明显钙化强回声，未见明显液化无回声。CFI可显示淋巴结淋巴门血管，较丰富。 穿刺记录:取仰卧位，充分暴露锁骨上，常规消毒，予7#穿刺细针，超声引导下穿刺，抽吸标本满意，涂片2张。穿刺点局部压迫后无出血。术后注意穿刺点情况。如有出血、穿刺点肿胀等严重者及时通知床位医生或护士。 诊断意见:A:右侧腋窝淋巴结穿刺2次，待细胞学结果B:右侧锁骨上淋巴结穿刺2次，待细胞学结果。

###异常检验指标
报告时间:2019-03-19 11:54:04
检验详情:INR:0.91,未判断;

报告时间:2019-03-19 12:48:52
检验详情:估算肾小球滤过率:121.2ml/min/1.73m2,未判断;

报告时间:2019-03-20 08:34:42
检验详情:乙肝病毒核心抗体:8.74(+),偏高; 乙肝病毒表面抗体:494.06(+)mIU/mL,偏高; 乙肝病毒e抗体:0.01(+),偏低;

报告时间:2019-03-20 14:50:56
检验详情:丙肝病毒抗体(HCV-Ab):阴性(-),未判断; 艾滋病毒抗体(HIV):阴性(-),未判断;

###全部检查
报告时间:2019-03-19 12:20:29
描述:心电图|常规心电图|心电图|
图像分析:窦性心律不齐

报告时间:2019-03-19 14:27:08
描述:放射|胸部|X-ray平片|胸部 胸部
图像所见:所示胸廓骨骼及胸壁软组织未见异常。 纵隔及气管居中未见移位。 纵隔未见增宽。 心脏形态大小未见异常。 两膈光整，两肋膈角锐利。 肺门形态大小位置未见异常。 两肺纹理略增多。
图像分析:两肺纹理略多。请结合临床及其他相关检查。

###最后一个在院评估单
###在院评估单:
时间:2019-03-19
基础评估:1.生活自理能力评估: Barthel指数总分60分。2.活动能力评估: 步态因卧床无法评估 ；活动状态:卧床 ，能自行翻身 ，移动患者时，摩擦力和剪切力无明显问题（可独立在床上或椅子上移动，移动时有足够的肌力将身体抬高，坐在椅子或床上随时都可以维持良好的体位） 。 3.意识状况评估:神志: 清醒 ，感知觉未受损（对指令有反应，能准确表达疼痛和不适） 。 4.粘膜与皮肤状况评估:颜色: 正常 ；温度: 正常 ；湿度: 正常（皮肤经常保持干燥，只需常规更换床单） ；皮肤完整性: 不完整 ，皮肤不完整原因切口 ，切口部位 胸部 。 无 水肿 ；无 静脉炎 ；无 瘙痒；无 麻木；无 灼烧感 。5.疼痛: 有 疼痛，切口疼痛，呈间隙性疼痛，轻度（注意时，有觉察） ，疼痛评分1分 。 6.排泄: 无 失禁 。 7.意外事件评估: 未发生意外事件。 营养状况评估:良好(饮食摄入和平时一样，每餐有蛋白质摄入) ，无营养支持。 危重症程度评估: 无 严重器官系统功能不全。 

      
'''
all_jiancha=jiancha_source[jiancha_source.find('###全部检查')+8:jiancha_source.find('###最后一个在院评估单')]
all_jiancha=all_jiancha[:all_jiancha.find('###')]
jiancha_list=re.split(r'报告时间:',all_jiancha)
jiancha_list=list(filter(None,jiancha_list))
print(jiancha_list)

suifang_dic={}
suifang_suggestion=''

for item in jiancha_list:
    item = item[item.find('描述'):]
    flag, history = model.chat(tokenizer, "请根据检查结果中的‘图像分析’，判断下面的检查结果是否需要随访，请从['是'，'否']中选择一个输出，不要输出其他内容。\n "+item, history=[] ,temperature=0.01)
    
    desc_match = re.search(r'描述:(.*?)(?:\n|$)', item)
    if desc_match:
        description = desc_match.group(1).split('|')[-2]+'：'
        # description = description[:description.find(' ')]+'：'
    else:
        description = ''
    # print(description)
    jiancha_summary,history= model.chat(tokenizer,"你是一位专业医生，请将下面的检查进行总结，请直接输出总结出的图像异常的检查项目和检查结果，精炼语句，将输出控制在一句话，不要提出建议\n"+item, history=[] ,temperature=0.01)
    jiancha_summary = jiancha_summary.replace("，随诊","").replace("，随访","")
    print(jiancha_summary)
    jiancha_summary=description+ jiancha_summary
    flag, history = model.chat(tokenizer, "请判断下面的检查结果是否有异常，有异常输出‘是’,没有异常输出‘否’,请从['是'，'否']中选择一个输出，不要输出其他内容。\n "+ jiancha_summary, history=[] ,temperature=0.01)
    if '其他检查' in item or '，随诊' in item or'，随访' in item or '其他相关检查' in item :
        flag='是'
    if '明显异常' in jiancha_summary or '未见异常' in jiancha_summary or '未发现异常' in jiancha_summary:
        flag='否'
    print(flag)
    

    if '是' in flag or flag=='需要随访。':
        rjkeshi_list="，请从下面的科室列表中选择随访科室:[‘心内科’,‘呼吸科’,‘乳腺外科’,‘胰腺外科’,‘消化科’,‘高血压科’,‘骨科’,‘妇科’,‘血液科’,‘耳鼻喉科’,‘甲状腺血管科’,‘神经内科’,‘神经外科’,‘肾脏内科’,‘内分泌科’,‘胃肠外科’,‘小儿科’]"
        suifang_keshi,history= model.chat(tokenizer,"你是一位专业医生，请根据下面的检查给出对应的随访科室"+rjkeshi_list+"，请最后输出唯一随访科室名称，例如：某某科，不要输出其他内容。\n"+item, history=[] ,temperature=0.01)
        print(suifang_keshi)
        if(suifang_dic.get(suifang_keshi,0)):
            suifang_dic[suifang_keshi]+=jiancha_summary
        else:
            suifang_dic[suifang_keshi]=jiancha_summary
        # jiancha_summary=jiancha_summary[:-1]
        # suifang_keshi='，建议于'+suifang_keshi+'随访。'
        # print("患者"+jiancha_summary+suifang_keshi)

for i in suifang_dic:
    jiancha_suggestion=suifang_dic[i]
    if jiancha_suggestion[-1]=='。':
        jiancha_suggestion=jiancha_suggestion[:-1]
    keshi_suggestion='，建议于'+i+'随访。'
    suifang_suggestion = '{}{}'.format(suifang_suggestion, "患者"+jiancha_suggestion+keshi_suggestion)


print(suifang_suggestion)

['2019-03-19 12:20:29\n描述:心电图|常规心电图|心电图|\n图像分析:窦性心律不齐\n\n', '2019-03-19 14:27:08\n描述:放射|胸部|X-ray平片|胸部 胸部\n图像所见:所示胸廓骨骼及胸壁软组织未见异常。 纵隔及气管居中未见移位。 纵隔未见增宽。 心脏形态大小未见异常。 两膈光整，两肋膈角锐利。 肺门形态大小位置未见异常。 两肺纹理略增多。\n图像分析:两肺纹理略多。请结合临床及其他相关检查。\n']
窦性心律不齐
是
心内科
两肺纹理略多。
是
呼吸科
患者心电图：窦性心律不齐，建议于心内科随访。患者X-ray平片：两肺纹理略多，建议于呼吸科随访。


In [122]:

def filter_kesuifang(item):
    return "科随访" not in item

suggestion='''1.10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案。\n1.患者窦性心动过缓，I°房室传导阻滞，建议于心血管内科随访。患者两肺纹理略多，左下肺小结节影，主动脉迂曲钙化，建议于呼吸科随访。
2.10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案。
2.术后胸带加压包扎切口5天；之后可佩戴文胸；保持伤口清洁干燥，2周内避免洗澡；术后10天内每3天来院换药一次，换药门诊时间：每周一～周五9:00-11:00 在门诊大楼3楼乳腺中心。
3.保持伤口清洁干燥，如有发热，切口局部红肿、疼痛、化脓等不适，及时来院就诊。
'''


pattern = r'[。|;][\n|''|' ']\d.' 
# suggestion,historty=model.chat(tokenizer,test_input, history=[] ,temperature=0.01)

suggestion_list=re.split(pattern,suggestion)
print(suggestion_list)
suggestion_list = list(filter(filter_kesuifang, suggestion_list))

final_suggestion=''
if re.match(r'\d\.',suggestion_list[0][:2]):
    suggestion_list[0]=suggestion_list[0][2:]
num=1

for s in suggestion_list:
    if num < len(suggestion_list):
        final_suggestion+=str(num)+'.'+s+'。\n'
    else:
        final_suggestion+=str(num)+'.'+s
    num+=1


print(final_suggestion)



['1.10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案', '患者窦性心动过缓，I°房室传导阻滞，建议于心血管内科随访。患者两肺纹理略多，左下肺小结节影，主动脉迂曲钙化，建议于呼吸科随访', '10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案', '术后胸带加压包扎切口5天；之后可佩戴文胸；保持伤口清洁干燥，2周内避免洗澡；术后10天内每3天来院换药一次，换药门诊时间：每周一～周五9:00-11:00 在门诊大楼3楼乳腺中心', '保持伤口清洁干燥，如有发热，切口局部红肿、疼痛、化脓等不适，及时来院就诊。\n']
1.10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案。
2.10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案。
3.术后胸带加压包扎切口5天；之后可佩戴文胸；保持伤口清洁干燥，2周内避免洗澡；术后10天内每3天来院换药一次，换药门诊时间：每周一～周五9:00-11:00 在门诊大楼3楼乳腺中心。
4.保持伤口清洁干燥，如有发热，切口局部红肿、疼痛、化脓等不适，及时来院就诊。



In [263]:
if '是'in flag:
    suifang_keshi,history= model.chat(tokenizer,"你是一位专业医生，请根据下面的检查给出对应的随访科室，请直接输出建议随访科室。请从下面的科室列表中选择随访科室:[‘呼吸科’,‘乳腺外科’,‘胰腺外科’,‘消化科’,‘高血压科’,‘骨科’,‘心内科’,‘妇科’,‘血液科’,‘肝胆外科’]\n"+jiancha, history=[] ,temperature=0.01)
    print(suifang_keshi)
    jiancha_summary,history= model.chat(tokenizer,"你是一位专业医生，请将下面的检查进行总结，请直接输出总结出的图像检查结果，不要提出建议\n"+jiancha, history=[] ,temperature=0.01)
    print(jiancha_summary)
    jiancha_summary=jiancha_summary[:-1]
    suifang_keshi='，建议于'+suifang_keshi+'随访。'

print("患者"+jiancha_summary+suifang_keshi)

乳腺外科
乳腺增强MRI：左乳头后方肿块伴坏死可能，炎症待排，建议穿刺活检。双乳腺病伴多发腺病结节，BI-RADS 3类。
患者乳腺增强MRI：左乳头后方肿块伴坏死可能，炎症待排，建议穿刺活检。双乳腺病伴多发腺病结节，BI-RADS 3类，建议于乳腺外科随访。


In [217]:

def input_all_jiancha(ICL,promote, question):
    return "{}\n{}\n\n检查结果:\n{}\n\n随访科室:".format(ICL,promote, question)

def compose_zs_prompt_jiancha(question,context):
    return "检查结果:\n{}\n\n随访建议：\n{}\n".format(question, context)

#例子
text1 = '''心电图提示：T波轻度改变'''

answer1 = '''患者心电图提示：T波轻度改变，建议于心内科随访。'''

text2 = '''胸部CT检查两肺多发小结节，左肺舌段索条'''

answer2 = '''患者胸部CT检查两肺多发小结节，左肺舌段索条，建议于呼吸科随访。'''

text3 = '''腹部超声提示：胆囊隆起样病变（考虑胆囊息肉）'''

answer3 = '''患者腹部超声提示：胆囊隆起样病变（考虑胆囊息肉），建议于肝胆外科随访。'''

promote = '''你是一位专业医生，请根据检查结果，参考上面例子，从科室列表中选择随访科室:["呼吸科","胰腺外科","消化科","高血压科","骨科","心内科","妇科","血液科","肝胆外科"]''' # ,并且通过自己的语言精简的描述小事件

ICL = compose_zs_prompt_jiancha(text1, answer1) + '\n' + compose_zs_prompt_jiancha(text2, answer2)+ '\n' + compose_zs_prompt_jiancha(text3, answer3)



prompt='''你是一位专业医生，参考上面的例子,请判断下面的患者的检查结果是否存在异常症状，如果有异常请给出科室随访建议(直接给出建议随访的科室)'''

jiancha= "###全部检查:\n报告时间:2019-01-03 12:49:45\n描述:放射|胸部|X-ray平片|胸部 胸部\n图像分析:两肺未见明显异常。请结合临床、病史及其他检查，随访。"

input=input_all_jiancha(ICL,prompt, jiancha)

response,history = model.chat(tokenizer,input, history=[] ,temperature=0.2)
print(response)


呼吸科


In [ ]:
test_input="###科室类别:\n胃肠外科\n\n###全部诊断:\n诊断时间:2023-01-07\n诊断类型:入院诊断\n诊断名称:直肠占位性病变\n\n诊断时间:2023-01-07\n诊断类型:入院诊断(补充)\n诊断名称:高血压病1级（高危）\n\n诊断时间:2023-01-09\n诊断类型:出院诊断\n诊断名称:直肠占位性病变\n\n诊断时间:2023-01-09\n诊断类型:出院诊断(补充)\n诊断名称:高血压病1级（高危）\n\n###入院记录:\n时间:2023-01-07\n主治医师48小时诊断:1、直肠占位性病变 2、高血压病1级（高危） 本人对于患者基本信息及现病史内容确认无误。 患者(委托人)手写:_____ 患者(委托人)签名:_____ 与患者关系:_____ 更正/补充诊断: 更正/补充日期: 医生签名:\n\n###主治医师首次查房记录:\n时间:2023-01-08\n诊断:1、直肠占位性病变 2、高血压病1级（高危）\n\n###主任医师首次查房记录:\n时间:2023-01-09\n补充病史与体征:神清，精神一般，无 皮疹及出血点，无 全身皮肤粘膜黄染，无 浅表淋巴结肿大，无 Virchow’sLN，无 脐周质硬结节。心率60 次/分，心律齐 ，无 闻及病理性杂音。腹平坦 ，全腹软 ，全腹无 压痛，无 反跳痛，无 腹部包块，无 肝脾肋下扪及，无 移动性浊音，Murphy's征（- ），肠鸣音正常 。无 双下肢水肿。NS（-）。 肛门指检:肛管直肠表面光滑，可及肿物\n对病情的分析:患者直肠指检于距肛门3-4cm触及肿物 。长期便秘，拟完善相关检查，判断肿物性质。\n诊疗意见:完善相关检查，根据结果制定诊疗方案，明日出院\n\n###出院带药:\n医嘱时间:2023-01-10\n医嘱类型名称:临时医嘱\t医嘱项名称:[出]杜密克(乳果糖口服溶液)[总:2瓶]\t医嘱项规格:200ml/瓶\t单次剂量数量:400.0\t单次给药数量:2.0\t给药途径:遵医嘱\t给药频次:ONCE\n\n医嘱时间:2023-01-10\n医嘱类型名称:临时医嘱\t医嘱项名称:[出]皿治林(咪唑斯汀缓释片)[总:2盒]\t医嘱项规格:10mg*7片/盒\t单次剂量数量:10.0\t单次给药数量:1.0\t给药途径:口服\t给药频次:ONCE\n\n###异常检验指标:\n报告时间:2023-01-07 18:10:58\n检验详情:新冠病毒ORF1ab基因:阴性(Neg),未判断;\t新冠病毒N基因:阴性(Neg),未判断;\t新型冠状病毒核酸检测:阴性(Neg),未判断;\n\n报告时间:2023-01-08 08:02:04\n检验详情:淋巴细胞%:18.6％,偏低;\t红细胞分布宽度:14.5％,偏高;\t血红蛋白:118g/L,偏低;\t红细胞比容:0.364,偏低;\t仪器名称:BC-6800_3,未判断;\t标本审核建议:Auto Validation OK,未判断;\n\n报告时间:2023-01-08 08:05:11\n检验详情:Fg:3.6g/L,偏高;\tINR:1.09,未判断;\n\n报告时间:2023-01-08 09:41:43\n检验详情:上皮细胞(镜检):0/HP,未判断;\t结晶(镜检):阴性(-),未判断;\t红细胞(镜检):0/HP,未判断;\t白细胞(镜检):0/HP,未判断;\n\n报告时间:2023-01-08 11:40:34\n检验详情:钙:1.99mmol/L,偏低;\t白球比例:1.19,偏低;\t白蛋白:32g/L,偏低;\t丙氨酸氨基转移酶:6IU/L,偏低;\t前白蛋白:162mg/L,偏低;\t胱抑素C:1.28mg/L,偏高;\t估算肾小球滤过率:56.2ml/min/1.73m2,未判断;\t钾:3.22mmol/L,偏低;\t总蛋白:59g/L,偏低;\n\n报告时间:2023-01-08 18:36:15\n检验详情:新冠病毒ORF1ab基因:阴性(Neg),未判断;\t新冠病毒N基因:阴性(Neg),未判断;\t新型冠状病毒核酸检测:阴性(Neg),未判断;\n\n报告时间:2023-01-09 11:20:56\n检验详情:新冠病毒ORF1ab基因:阴性(Neg),未判断;\t新冠病毒N基因:阴性(Neg),未判断;\t新型冠状病毒核酸检测:阴性(Neg),未判断;\n\n报告时间:2023-01-09 12:15:55\n检验详情:艾滋病毒抗体(HIV):阴性(-),未判断;\t丙肝病毒抗体(HCV-Ab):0.03(-),未判断;\n\n报告时间:2023-01-09 13:45:14\n检验详情:乙肝病毒e抗体:0.16(+),偏低;\t乙肝病毒表面抗体:295.10(+)mIU/mL,偏高;\t乙肝病毒核心抗体:6.52(+),偏高;\n\n报告时间:2023-01-09 14:40:36\n检验详情:丁型肝炎病毒抗原:阴性(-),未判断;\t丁型肝炎病毒抗体IgG:阴性(-),未判断;\t戊型肝炎病毒抗体IgG:阴性(-),未判断;\t戊型肝炎病毒抗体IgM:阴性(-),未判断;\n\n报告时间:2023-01-10 09:46:40\n检验详情:绿链及干燥奈瑟氏菌:,偏高;\n\n###全部检查:\n检查ID:2023010930000009|1\n报告时间:2023-01-09 13:56:27\n描述:内窥镜|电子肠镜|住院电子肠镜|电子肠镜 电子肠镜\n图像所见:备 注：进镜至回盲部，肠道准备结肠，肠腔内多处粪渣粪水影响视野。 回盲部：见唇形回盲瓣。 升结肠：未见明显异常。 肝 曲：外来紫色压迹可见。 横结肠：结构正常，血管网纹理清晰。 脾 曲：未见明显异常。 降结肠：未见明显异常。 乙结肠：未见明显异常。 直 肠：距肛3cm左右见菜花状病灶，占肠腔3/4周。质脆易出血，活检数枚。 肛 管：未见明显异常。\n图像分析:直肠占位\n\n检查ID:2023010700000930|3\n报告时间:2023-01-10 10:33:37\n描述:放射|增强|住院CT|胸部+上中腹部+下腹盆腔 增强\n图像所见:直肠肠壁黏膜异常增厚强化，肠腔狭窄，周围脂肪间隙模糊伴小结节影，多发小静脉影。乙状结肠系膜、肠系膜下静脉走行区淋巴结增大。 盆腔未见阳性结石影，未见异常钙化影。 膀胱充盈差，未见异常密度。 前列腺稍大，密度未见明显异常。 腹膜后小淋巴结影。\n图像分析:拟直肠MT，周围脂肪间隙模糊伴小结节影，周围静脉曲张，乙状结肠系膜、肠系膜下静脉走行区淋巴结增大。前列腺稍大。腹膜后小淋巴结影。请结合临床及其他检查，随诊。\n\n检查ID:2023010700000930|2\n报告时间:2023-01-10 10:33:37\n描述:放射|增强|住院CT|胸部+上中腹部+下腹盆腔 增强\n图像所见:肝脏形态大小正常，肝内多发无强化囊性低密度影。 胆囊未见明显异常。 脾脏形态大小正常，未见异常密度影。 胰腺形态大小正常，胰头密度减低。 两肾未见异常。 腹膜后小淋巴结影。 阑尾略粗，腔内高密度影。\n图像分析:肝多发囊肿。胰头脂肪浸润改变。阑尾略粗，粪石形成。腹膜后小淋巴结影。请结合临床及其他相关检查，随诊。\n\n检查ID:2023010700000930|1\n报告时间:2023-01-10 10:33:37\n描述:放射|增强|住院CT|胸部+上中腹部+下腹盆腔 增强\n图像所见:左肺下叶外基底段（SE402，IM36）见一枚磨玻璃结节，长径约4mm。 左肺上叶尖后段（SE402，IM20）见一枚实性结节，长径约4mm。 右肺上叶尖段（SE402，IM18）见一枚实性结节，长径约3mm。 右肺下叶前基底段（SE402，IM38）见一枚实性结节，长径约3mm。 两肺尖胸膜稍厚。 两肺门区未见异常密度影。 主动脉，肺动脉主干及其左右分支内造影剂密度均匀。各大血管边界清晰。 纵隔内淋巴结影显示。\n图像分析:两肺微小结节。两肺尖胸膜稍厚。纵隔内淋巴结影显示。请结合临床及其他相关检查，随诊。\n\n检查ID:2023010910003751|1\n报告时间:2023-01-10 12:57:08\n描述:放射|增强|住院MRI|直肠-MR 增强\n图像所见:肿块特点：以稍低信号为主的混杂信号，增强轻中度强化； 肿块位置：位于直肠中下段，肿瘤最下端距离肛缘约4.2cm，位于腹膜反折以下； 肿块大小：约累及肠管长约5.1cm；约累及1周； T：肿瘤浸润至肌层外脂肪组织； N：直肠系膜内见数枚淋巴结显示，其中约3枚边界欠清，信号欠均，较大者截面大小约0.5*0.4cm；右侧侧方2枚淋巴结显示；双侧腹股沟未见明显肿大淋巴结影； MRF：受侵，为肿瘤直接侵犯； EMVI：2分。 附见：前列腺肿大，局部凸向膀胱；膀胱壁增厚；前列腺信号减低欠均，左侧团块状低信号影，弥散受限，ADC减低，增强轻度不均匀强化。\n图像分析:直肠中下段占位，浸润至肌层外脂肪组织，拟直肠癌（mrT3aN1）病变肠段局部与前列腺外围带关系密切；MRF（+）；EMVI（-）；右侧侧方2枚淋巴结显示；双侧腹股沟未见明显肿大淋巴结影；请结合临床病史及肠镜检查。附见：前列腺肿大，信号减低欠均；膀胱充盈欠佳。\n\n检查ID:2023010910003841|1\n报告时间:2023-01-10 13:00:18\n描述:放射|平扫|住院MRI|直肠-MR 平扫\n图像所见:距肛缘约4.2cm直肠中下段约5.1cm肠段近肠壁1周不规则增厚，肠壁欠光整，跨腹膜反折生长，病灶T2W呈稍低信号，信号欠均，DWI明显受限，ADC信号减低；浸润至固有肌层外脂肪组织可能，肠系膜内见数枚淋巴结显示，部分边界不清，信号欠均匀；右侧侧方2枚淋巴结显示；双侧腹股沟未见明显肿大淋巴结影；余所示骨盆骨质信号未见明显异常。周围软组织未见明显异常信号影。附见：前列腺肿大，局部凸向膀胱；膀胱壁增厚；前列腺信号减低欠均，左侧团块状低信号影，弥散受限，ADC减低。\n图像分析:直肠中下段占位，浸润至固有肌层外脂肪组织可能，系膜内多枚淋巴结显示，部分边界欠清，信号不均；右侧侧方2枚淋巴结显示；请结合临床病史及相关检查，随访。附见：前列腺肿大，局部凸向膀胱；膀胱壁增厚；前列腺信号减低欠均伴左侧团块状低信号影。\n\n###未出检查结果报告:\n无\n\n###未出结果病理报告:\n\n\n检查时间:2023-01-10\n临床诊断:直肠占位性病变\n病理类型:常规\n\n###最后一个在院评估单:\n###在院评估单:\n时间:2023-01-10\n基础评估:1.生活自理能力评估: Barthel指数总分100分。 。2.活动能力评估: 步态稳定 ；活动状态:活动自如 。 3.意识状况评估:神志: 清醒 ，感知觉未受损（对指令有反应，能准确表达疼痛和不适） 。 4.粘膜与皮肤状况评估:颜色: 正常 ；温度: 正常 ；湿度: 正常（皮肤经常保持干燥，只需常规更换床单） ；皮肤完整性: 完整 。 无 水肿 ；无 静脉炎 ；无 瘙痒；无 麻木；无 灼烧感 。5.疼痛: 无 疼痛 。 6.排泄: 无 失禁 。 7.意外事件评估: 未发生意外事件。 患者昨夜睡眠: 正常 营养状况评估:良好(饮食摄入和平时一样，每餐有蛋白质摄入) ，无营养支持。 危重症程度评估: 无 严重器官系统功能不全。 无 免疫损害。 格拉斯评分: 15分 。 危重症评分: 5分 。 高危因素的筛查 该病人目前不存在跌倒风险 该病人目前不存在压疮风险。 分级护理级别: 该病人分级护理建议为三级护理。\n置管状态:无置管。 【医学元素\\签名\\护士签名2-护理评估】 【医学元素\\签名\\护士签名3-护理评估】\n请参照患者的医疗资料，编写其出院小结文书的“出院后用药建议”字段部分。请按标准的文本格式输出。\n字段介绍:输出结果应以数字化的清单形式展示，每条用药建议前标明数字编号，如‘1.’, ‘2.’, ‘3.’，确保信息条理清晰。\n撰写建议:请提醒患者出院后的注意事项；根据患者的手术过程提醒患者对伤口与拆线的注意事项，与部分手术可能造成的术后情况；描述患者的出院带药。"
response,history= model.chat(tokenizer,test_input, history=[] ,temperature=0.01)
print(response)


1、注意休息，避免劳累，饮食以软食为主，忌辛辣刺激食物，忌烟酒。 2、出院后定期复查血常规、肝肾功能、电解质、肿瘤指标等。 3、如有腹痛、发热、便血、黑便等不适，及时就诊。 4、出院带药：杜密克（200ml*2瓶），口服，每日2次，每次20ml。 5、出院后1月左右至我院门诊随访，根据病理结果制定下一步诊疗计划。


In [183]:

def input_all(ICL, promote1, promote2, question):
    return "{}\n{}\n{}\n\n患者情况:\n{}\n\n病史总结:".format(ICL, promote1, promote2, question)

text1 = '''疾病史: 高血压病病史15年，血压最高为不详（患者已忘记，无法描述），口服“替米沙坦胶囊80mg qm，苯磺酸左旋氨氯地平片1片 qm”治疗，平日血压控制在130/70mmHg左右。1997年起血小板减少症，自1997年起至今服用强的松1#qd，2019.01.04日复查血常规示血小板20×10^9/L。自述早搏5年，2018年4月住院期间行动态心电图示:房性早搏321次，其中短阵房速5阵；多源室性早搏71次；完全性右束支传导阻滞；ST发作性改变。患者自述偶有心慌感，心慌时自服保心丸后可缓解，现口服稳心颗粒1包 qd治疗。否认糖尿病、冠心病、COPD等慢性疾病史。 传染病史: 否认 传染性疾病史。'''
answer1 = '''疾病：高血压,血小板减少症,早搏,心慌'''
text2 = '''疾病史: 否认高血压、糖尿病、冠心病、COPD等慢性疾病史。 传染病史: 承认 传染性疾病史，乙肝大三阳20余年，目前服用恩替卡韦1'''
answer2 = '''疾病：乙肝'''
text3 = '''疾病史: 否认高血压、糖尿病、冠心病、COPD等慢性疾病史 传染病史: 承认或否认 传染性疾病史，疾病名称。'''
answer3 = '''疾病：无'''

# ICL = compose_zs_prompt_1(text1, answer1) + '\n' + compose_zs_prompt_1(text2, answer2) + '\n' + compose_zs_prompt_1(text3, answer3)

test_input1="疾病史: 承认高血压、糖尿病、冠心病、COPD史。患者没有贫血史。 "
test_input="疾病史: 癫痫病史，曾服用苯妥英钠6年治疗，后病情稳定，未再复发，已停药30余年。脑血管疾病、房颤、高血压病史，服用蒙诺0.5片#qd、施慧达1#qd、特TM员工名称TM1#qd、螺内酯1#qd、欣康1#qd、芪苈强心3#tid治疗，口服拜瑞妥，已停药6天。否认糖尿病、COPD等慢性疾病史。"
# test_input = "一般健康状况: 身体健康 疾病史: 否认高血压、糖尿病、冠心病、COPD等慢性疾病史。患者有贫血史。 传染病史: 否认 传染性疾病史。 预防接种史: 按计划免疫接种。 手术外伤史: 承认 手术史，1993年行剖腹产，否认外伤史。 输血史: 否认输血史 食物过敏史: 否认食物过敏史 药物过敏史: 否认药物过敏史"
test_input=test_input.replace("否认","没有")
promote1 = '''疾病列表:["乙肝","心肌炎","桥本甲状腺炎","高血压","甲状腺结节","心绞痛","早搏","糖尿病","过敏性支气管炎","肾盂肾炎","子宫肌瘤","肾结石","贫血","垂体微腺瘤","鼻炎","湿疹","甲亢","脑梗","心肌缺血","白内障","鼻窦炎","腰椎间盘突出","胸腺瘤"]''' 
promote2 = '''你是一位专业医生，根据患者情况，判断患者是否存在疾病列表中的疾病,存在就输出疾病列表中的疾病，无匹配项就不输出疾病，注意否认高血压、糖尿病、冠心病、COPD等慢性疾病史不应该输出疾病。'''

def input_all(ICL, promote1, promote2, question):
    return "{}\n{}\n{}\n\n患者情况:\n{}\n\n病史总结:".format(ICL, promote1, promote2, question)

# response, history = model.chat(tokenizer, input_all(ICL, promote1, promote2, value), history=[], temperature=0.2)



flag2,history= model.chat(tokenizer,"你是一位专业医生，请根据下面的患者的病史情况描述，判断患者是否有病史，有病史就直接输出‘有’，并输出每个病史，每个病史单独输出，不要输出其他内容。\n"+test_input, history=[] ,temperature=0.01)
print(flag2)

# response,history= model.chat(tokenizer,"你是一位专业医生，请根据下面的患者的病史情况描述，输出每个病史需要随访的对应的科室，随访的科室是唯一且最恰当的科室，每个病史和对应科室单独输出，输出格式为【病史：科室】，不要输出其他内容。\n"+flag2, history=[] ,temperature=0.01)
# print(response)

有，病史如下：
1. 癫痫病史：曾服用苯妥英钠6年治疗，后病情稳定，未再复发，已停药30余年。
2. 脑血管疾病史：有。
3. 房颤史：有。
4. 高血压病史：有。
5. 服用蒙诺0.5片#qd、施慧达1#qd、特TM员工名称TM1#qd、螺内酯1#qd、欣康1#qd、芪苈强心3#tid治疗史：有。
6. 口服拜瑞妥史：有。
7. 停药6天：有。
8. 没有糖尿病、COPD等慢性疾病史：有。


In [80]:
result1='''
1.患者窦性心动过缓，I°房室传导阻滞，建议于心血管内科随访。患者两肺纹理略多，左下肺小结节影，主动脉迂曲钙化，建议于呼吸科随访。
2.10个工作日后来院询问石蜡病理报告，并带至主诊医师门诊就诊，拟定进一步诊疗或随访方案。
2.术后胸带加压包扎切口5天；之后可佩戴文胸；保持伤口清洁干燥，2周内避免洗澡；术后10天内每3天来院换药一次，换药门诊时间：每周一～周五9:00-11:00 在门诊大楼3楼乳腺中心。
3.保持伤口清洁干燥，如有发热，切口局部红肿、疼痛、化脓等不适，及时来院就诊。
'''
conclude="请你帮我重新给每个句子编号，编号从1开始，不要改写文本，直接输出编号整理好的文本\n需要编号的文本："

conclude_suggestion,history= model.chat(tokenizer,conclude+result1, history=[] ,temperature=0.01)
print(conclude_suggestion)

KeyboardInterrupt: 

In [30]:
text='''
患者入院后完善相关检查,并于2019-03-18全麻下行左乳肿物麦默通旋切术,术顺安返,术后予以补液支持治疗。现患者一般情况
可,伤口愈合I/甲,经上级医师查房同意后,予以今日出院。2019-03-18石蜡病理报告未出。术后病理报告示:乳腺良性肿瘤
(左乳,待石蜡)。术后石蜡病理报告未出。目前患者一般情况可,伤口愈合I/甲,经上级医师查房同意后,予以出院。
'''
conclude1,conclude2,conclude3,conclude4,conclude5='综合分析患者的医疗记录，包括手术、化疗、报告和当前情况等，以一段文本总结患者住院期间的整体治疗进展。','从提供的医疗信息中，详细梳理并总结患者在住院期间接受的所有治疗措施，包括手术、化疗、报告及当前情况等。','分析患者的住院病历，提取并概述其在住院期间经历的治疗过程，如手术、化疗、报告和当前情况等。','处理并总结患者在住院期间的病程信息，关注手术、化疗、报告以及和当前情况等关键治疗环节。','对患者的住院病程数据进行分析，集中呈现其在医院期间接受的主要治疗方式，包括手术、化疗、报告及当前情况等信息。'
conclude="请帮我删除下面这段文本中重复的内容和句子，然后直接输出删除过的文本，不要输出其他内容\n文本:\n"

conclusion,history= model.chat(tokenizer,conclude+text, history=[] ,temperature=0.01)
print(conclusion)

患者入院后完善相关检查,并于2019-03-18全麻下行左乳肿物麦默通旋切术,术顺安返,术后予以补液支持治疗。现患者一般情况可,伤口愈合I/甲,经上级医师查房同意后,予以今日出院。2019-03-18石蜡病理报告未出。术后病理报告示:乳腺良性肿瘤(左乳,待石蜡)。术后石蜡病理报告未出。目前患者一般情况可,伤口愈合I/甲,经上级医师查房同意后,予以出院。
